In [1]:
import pandas as pd
from pythonmodules.config import Config
from sqlalchemy import create_engine
from pythonmodules.namenlijst import Namenlijst
from pythonmodules.mediahaven import MediaHaven
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

In [2]:
conf = Config()
con = create_engine(conf['db']['connection_url'])
nml = Namenlijst()
mh = MediaHaven()
mapping = dict(zip(range(0, 4), ['Undefined', 'Matches', 'No match', 'Uncertain']))

In [3]:
df = pd.read_sql(sql='SELECT nmlid, pid, status, entity, kind, extras FROM attestation_link WHERE status != 0', con=con)
df.set_index('nmlid', inplace=True)

In [4]:
nml_data = [next(nml.findPerson(document={"_id": nmlid})) for nmlid in df.index]

In [5]:
df = pd.DataFrame(nml_data).set_index('_id').join(df.replace({'status': mapping}))

In [6]:
# more efficient way, but gives 414 Request-URI Too Large
# df_mh = mh.search('+(%s)' % ' '.join('externalId:' + pid for pid in df['pid']), 0, 1)
data_mh = (next(mh.search('+(externalId:%s)' % pid, 0, 1)) for pid in df['pid'])

In [7]:
texts = []
df_mh = []
for item in tqdm(data_mh, total=len(df)):
    texts.append({
        'pid': item['externalId'],
        'text': item['description']
    })
    data = {
        'pid': item['externalId'],
        'attributes': {
            attr['attribute']: attr['value'] for attr in item['mdProperties'] if attr['attribute'] != 'premis'
        },
        'title': item['title'],
        'previewImagePath': item['previewImagePath'],
    }
    for k, v in data['attributes'].items():
        data[k] = v
    del data['attributes']
    del data['status']
    df_mh.append(data)

df_mh = pd.DataFrame(df_mh).set_index('pid')
del data_mh


217it [01:11,  3.05it/s]


In [8]:
df = df.join(df_mh, on='pid', rsuffix='_mh')
df.columns

Index(['alternative_familynames', 'alternative_surnames', 'born_day',
       'born_month', 'born_year', 'cwxrm_remembered', 'description',
       'died_age', 'died_day', 'died_month', 'died_year', 'familyname',
       'gender', 'in_namelist', 'initials', 'memorials', 'nationality',
       'project_memberships', 'relations', 'sort_born_date', 'sort_died_date',
       'surname', 'victim_type', 'victim_type_details', 'war_casualty', 'pid',
       'status', 'entity', 'kind', 'extras', 'CP', 'CP_id', 'CreationDate',
       'PID', 'RightsOwner', 'Sub_CP', 'abraham_ID', 'batch_id', 'batch_name',
       'carrier_barcode', 'carrier_date', 'carrier_unity',
       'collection_box_barcode', 'cover_and_bindings', 'created_by',
       'created_on', 'digitization_format', 'dimensions', 'edition',
       'language', 'number', 'number_of_pages', 'original_carrier_id',
       'original_location', 'paper', 'preservation_problems',
       'previewImagePath', 'shipment_id', 'sp_id', 'sp_name',
       'text

In [9]:
#items = ['victim_type_details', 'victim_type', 'pid', ]
#fig, axs = plt.subplots(nrows=len(items))
#for idx, t in enumerate(items):
#    sns.countplot(y='status', data=df, hue=t, ax=axs[idx])

In [10]:
counts = df.groupby(['victim_type', 'status'])['pid'].count()
counts

victim_type  status   
CIVILIAN     Matches        5
             No match      14
             Uncertain      2
MILITARY     Matches       19
             No match     177
             Uncertain     30
Name: pid, dtype: int64

In [11]:
counts = df['entity'].value_counts()
freq_entities = counts.head(5).index.tolist()
# ['status'].value_counts()
# df['entity'] in counts['entity']
counts = df[df['entity'].isin(freq_entities)].groupby(['entity', 'kind'])['pid'].count()
counts

entity             kind           
Albert Thomas      Artikel            10
                   Correspondentie     1
Charles Bernard    Artikel             4
                   namenlijst          4
Emile Vandervelde  Artikel             4
                   rechtszaken         4
George Lloyd       Artikel            17
Louis Nicolas      Artikel             1
                   Lijst               3
                   lijst soldaten      3
Name: pid, dtype: int64

In [12]:
df.groupby(['status']).count()

,alternative_familynames,alternative_surnames,born_day,born_month,born_year,cwxrm_remembered,description,died_age,died_day,died_month,...,previewImagePath,shipment_id,sp_id,sp_name,text_quality_for_ocr,text_type,title,type_viaa,updated_by,updated_on
status,,,,,,,,,,,,,,,,,,,,,
Matches,24,24,12,12,12,24,24,24,22,22,...,24,24,24,24,24,16,24,24,24,24
No match,191,191,104,105,109,191,191,191,187,187,...,191,191,191,191,191,151,191,191,191,191
Uncertain,32,32,16,16,16,32,32,32,30,30,...,32,32,32,32,32,31,32,32,32,32
